In [ ]:
# pip install tensorflow

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [ ]:
df = pd.read_csv(r'Tweets.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
tweet_df = df[['text','airline_sentiment']]
print(tweet_df.shape)
tweet_df.head()
# tweet_df = tweet_df[tweet_df['airline_sentiment'] != 'positive']
# print(tweet_df.shape)
# tweet_df.head()

In [ ]:
tweet_df["airline_sentiment"].value_counts()

In [ ]:
sentiment_label = tweet_df.airline_sentiment.factorize()
sentiment_label

In [ ]:
tweet = tweet_df.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [ ]:
print(tokenizer.word_index)

In [ ]:
print(tweet[2])
print(encoded_docs[2])

In [ ]:
print(padded_sequence[0])

In [ ]:
from keras.layers import LeakyReLU

In [ ]:
import tensorflow as tf

In [ ]:
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(120, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.3))
model.add(Dense(3)) 
model.add(LeakyReLU(alpha= 0.1))
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=15, batch_size=32)

In [ ]:
model.save("sentiment_tweets_15_Epochs.h5")
print("Model saved")

In [ ]:
from numpy import loadtxt
from tensorflow.keras.models import load_model
 
# load model
model1 = load_model('sentiment_tweets_15_Epochs.h5')
# summarize model.
model1.summary()

# dataset = loadtxt("Tweets.csv", delimiter=",")
# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# score = model1.evaluate(X, Y, verbose=0)

In [ ]:
result=model.evaluate(padded_sequence, sentiment_label[0], batch_size=32)

In [ ]:
plt.plot(model.history.history['accuracy'], label='acc')
plt.plot(model.history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()
plt.savefig("Accuracy plot.jpg")
# model1.history.history

# plt.plot(history.history['accuracy'], label='acc')
# plt.plot(history.history['val_accuracy'], label='val_acc')
# plt.legend()
# plt.show()
# plt.savefig("Accuracy plot.jpg")

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig("Loss plot.jpg")

In [ ]:
import numpy as np
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = model1.predict(tw)
    
    prediction.astype(int)
    
    thisdict = {
      0: "neutral",
      1: "positive",
      2: "negative"
    }
    
    print(thisdict[np.argmax(prediction)])

In [ ]:
predict_sentiment("This is fine.")

predict_sentiment("This is the worst experience of my life.")

predict_sentiment("This is best!")